In [21]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
env=AngryBird()

# 240 coeffs

In [22]:
seconds = np.arange(11)
pos = np.zeros((11, 2))
pos[:, 0] = seconds * 10
pos[:, 1] = seconds * 20 - 2 * seconds**2

In [23]:
mean_states = np.load("mean_states.npy")

In [24]:
cor = np.array([[0, 0, 0],
               [0, 3, 2],
               [0, 1, 0]])

In [27]:
coeffs = np.load("cc.npy")
coeffs2 = np.load("cc2.npy")

In [28]:
def main(state, coeffs, coeffs2):
    t = int(state[0])
    a, b =np.sign(state[1:] - mean_states[t]).astype(int)
    i = cor[a][b]
    if t < 9:
        return coeffs[t, i, :2]*pos[t]-coeffs[t, i, 2:4]*state[1:] + coeffs[t, i, 4:]
    else:
        return coeffs[t, i, :2]*pos[t]-coeffs[t, i, 2:4]*state[1:] + coeffs[t, i, 4:] + coeffs2*state[1:]**2 * (i < 0.5)

In [16]:
from time import time
nul=np.zeros(2)
m = 100
n = 200
vents = np.load("vents.npy")
ecart = 0.0001
cos_min = 400.
while cos_min > 100:
    t = -time()
    '''
    for i in range(10):
        for l in range(4):
            for k in range(6):
                coeff_change = np.linspace(coeffs[i][l][k]-ecart, coeffs[i][l][k]+ecart, 9)
                costs = np.zeros(9)
                for j in range(9):
                    coeffs[i][l][k] = coeff_change[j]
                    for z in range(m):
                        state = env.reset()
                        env.vent = vents[z]
                        for _ in range(10):
                            force = main(state, coeffs, coeffs2)
                            state, _, _, _ = env.step(force)
                        env.step(nul)
                        costs[j] += env.cost.sum()
                coeffs[i][l][k] = coeff_change[np.argmin(costs)]
    '''
    for k in range(2):
        coeff_change = np.linspace(coeffs2[k]-ecart, coeffs2[k]+ecart, 9)
        costs = np.zeros(9)
        for j in range(9):
            coeffs2[k] = coeff_change[j]
            for z in range(m):
                state = env.reset()
                env.vent = vents[z]
                for _ in range(10):
                    force = main(state, coeffs, coeffs2)
                    state, _, _, _ = env.step(force)
                env.step(nul)
                costs[j] += env.cost.sum()
        coeffs2[k] = coeff_change[np.argmin(costs)]

    cos = 0.
    for z in range(n):
        state = env.reset(False)
        env.vent = vents[z]
        for _ in range(10):
            force = main(state, coeffs, coeffs2)
            state, _, _, _ = env.step(force)
        env.step(nul)
        cos += env.cost.sum()
    cos /= n
    t += time()
    print(m, n, ecart, costs.min()/m, cos, t/60)
    if cos < cos_min:
        cos_min = cos
        np.save("cc", coeffs)
        np.save("cc2", coeffs2)
    else:
        cos_min = cos + 20
        m += 100
        n += 100
        ecart /= 2

100 200 0.0001 109.99319921025433 116.31893628115569 0.0789316217104594
100 200 0.0001 109.99319921025433 116.31893628115569 0.07893474499384562
200 300 5e-05 116.3186529468785 113.33214761268687 0.15360819101333617
200 300 5e-05 116.318449636093 113.33211124464374 0.15436457395553588
200 300 5e-05 116.31832634879933 113.33213495376724 0.15469741423924763
300 400 2.5e-05 113.33211558955969 121.01998579642166 0.23109429677327473
300 400 2.5e-05 113.33211092281417 121.0199930965694 0.2296617309252421
400 500 1.25e-05 121.01998736244313 117.13749113131611 0.3049569845199585
400 500 1.25e-05 121.01998509776138 117.13746423354954 0.30423769156138103
400 500 1.25e-05 121.01998507369164 117.13745801839585 0.30494961738586424
400 500 1.25e-05 121.01998507369164 117.13745801839585 0.3038249731063843
500 600 6.25e-06 117.13728988019442 120.25712081050901 0.3797134757041931
500 600 6.25e-06 117.13727976609506 120.2571094418784 0.3792426387468974
500 600 6.25e-06 117.13727046685649 120.25709888456

KeyboardInterrupt: 

In [44]:
n = 10000
cos = 0.
X = []
Y = []
for z in range(n):
    state = env.reset(False)
    env.vent = vents[z]
    for y in range(9):
        force = main(state, coeffs, coeffs2)
        state, _, _, _ = env.step(force)
    force = main(state, coeffs, coeffs2)
    X.append(np.copy(state[1:]))
    state, _, _, _ = env.step(force)
    Y.append(-state[1:])
X = np.array(X)
Y = np.array(Y)

In [52]:
import tensorflow as tf
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(10, input_shape=(2,)))
        self.model.add(tf.keras.layers.Dense(10))
        self.model.add(tf.keras.layers.Dense(2))
        self.model.compile(optimizer='adam', loss='mse')
    def fit(self, X, Y):
        es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
        self.model.fit(X, Y, batch_size=32, epochs=200, verbose=1, callbacks=[es])
    def predict(self, x):
        return self.model.predict(x.reshape(1, 2))
    def save(self):
        self.save_weights("model")

In [53]:
model = Model()

In [54]:
model.fit(X, Y)

Epoch 1/200
313/313 [==============================] - 1s 3ms/step - loss: 63.2275
Epoch 2/200
313/313 [==============================] - 1s 3ms/step - loss: 3.6408
Epoch 3/200
313/313 [==============================] - 1s 3ms/step - loss: 2.4174
Epoch 4/200
313/313 [==============================] - 1s 3ms/step - loss: 1.3836
Epoch 5/200
313/313 [==============================] - 1s 3ms/step - loss: 0.8485
Epoch 6/200
313/313 [==============================] - 1s 3ms/step - loss: 0.6920
Epoch 7/200
313/313 [==============================] - 1s 3ms/step - loss: 0.6698
Epoch 8/200
313/313 [==============================] - 1s 3ms/step - loss: 0.6695
Epoch 9/200
313/313 [==============================] - 1s 3ms/step - loss: 0.6738
Epoch 10/200
313/313 [==============================] - 1s 3ms/step - loss: 0.6719
Epoch 11/200
313/313 [==============================] - 1s 3ms/step - loss: 0.6735
Epoch 12/200
313/313 [==============================] - 1s 3ms/step - loss: 0.6828
Epoch 13/200

In [55]:
model.save()